# 10. 聚类

## 10.1. 聚类任务
在无监督学习(unsupervised learning)中，训练样本的标记信息是未知的，目标是通过对无标记训练样本的学习来揭示数据的内在性质及规律，为进一步的数据分析提供基础。此类学习任务中研究最多、应用最广的是“聚类”(clustering).

聚类试图将数据集中的样本划分为若干个通常是不相交的子集，每个子集被称为一个“簇”(cluster)。通过这样的划分，每个簇可能对应于一些潜在的概念(类别)，例如“浅色瓜”、“深色瓜”等。**需要说明的是，这些概念对聚类算法而言事先是未知的，聚类过程仅能自动形成簇，簇所对应的概念语义需要使用者来把握和命名**

聚类既能作为一个单独的过程，用于寻找数据内在的分布结构，也可作为分类等其他学习任务的前驱过程。例如，在一些商业应用中需对新用户的类型进行判别，但定义“用户类型”对商家来说可能不太容易，此时可先对用户数据进行聚类，根据聚类结果将每个簇定义为一个类，然后再基于这些类训练分类模型，用于判别新用户的类型。

## 10.2. 性能度量
聚类的性能度量大致有两类。一类是将聚类结果与某个“参考模型”(reference model)进行比较，称为“外部指标”(external index);另一类是直接考察聚类结果而不利用任何参考模型，称为“内部指标”(internal index)。

### 10.2.1. 外部指标

对数据集$D=\{x_1,x_2,...,x_m\}$，假定通过聚类给出的簇划分为$C=\{C_1,C2,...,C_k\}$，参考模型给出的簇划分为$C^*=\{C_1^*,C_2^*...,C_s^*\}$(通常$k\neq s$)。相应的，令$\lambda$与$\lambda^*$分别表示$C$与$C^*$对应的簇标记向量。将样本两两配对考虑，定义：

$
a = |SS|, SS = \{(x_i,x_j)|\lambda_i=\lambda_j, \lambda_i^* = \lambda_j^*, i<j\} \\
b = |SD|, SD = \{(x_i,x_j)|\lambda_i=\lambda_j, \lambda_i^* \neq \lambda_j^*, i<j\} \\
c = |DS|, SS = \{(x_i,x_j)|\lambda_i \neq \lambda_j, \lambda_i^* = \lambda_j^*, i<j\} \\
d = |DD|, DD = \{(x_i,x_j)|\lambda_i \neq \lambda_j, \lambda_i^* \neq \lambda_j^*, i<j\}
$

集合SS包含了在C中隶属相同簇且在$C^*$中也隶属相同簇的样本对，集合SD包含了在C中隶属相同簇但在$C^*$中隶属与不同簇的样本对。。。。。

由于每个样本对$(x_i, x_j) (i<j)$仅能出现在一个集合中，因此有$a+b+c+d=m(m-1)/2$成立

基于上面的4个式子，可以导出常用的聚类性能度量外部指标：

- Jaccard系数（Jaccard Coefficient, 简称JC）

$
JC = \frac{a}{a+b+c}
$

- FM指数（Fowlkes and Mallows Index, 简称FMI）

$
FMI = \sqrt{\frac{a}{a+b} \cdot \frac{a}{a+c}}
$

- Rand指数（Rand Index，简称RI）

$
RI = \frac{2(a+b)}{m(m-1)}
$ 

上述性能指标的结果均在[0,1]区间，值越大越好

### 10.2.2. 内部指标

考虑聚类结果的簇划分$C=\{C_1,C_2,...,C_k\}$，定义：

$
avg(C) = \frac{2}{|C|(|C|-1)} \sum_{1 \le i \le j \le |C|}dist(x_i,x_j) \\
diam(C) = \max_{1 \le i \le j \le |C|}dist(x_i,x_j) \\
d_{\min}(C_i, C_j) = \min_{x_i \in C_i,x_j \in C_j} dist(x_i,x_j) \\
d_{cen}(C_i, C_j) = dist(\mu_i, \mu_j)
$

其中dist()用于计算两个样本之间的距离；$\mu$代表簇C的中心点$\mu=\frac{1}{|C|}\sum_{1 \le i \le |C|}x_1$。显然avg(C)对应于簇C内样本间的平均距离，diam(C)对应于簇C内样本间的最远距离，$d_{\min}(C_i, C_j)$对应于簇$C_i$与簇$C_j$最近样本距离，$d_{cen}(C_i, C_j)$对应于簇$C_i$与簇$C_j$中心点间的距离。

基于上面的4个式子，可导出常用的内部指标：

- DB指数（Davies-Bouldin Index，简称DBI）

$
DBI = \frac{1}{k}\sum\limits_{i=1}^k \max\limits_{j \neq i} \big(\frac{avg(C_i)+avg(C_j)}{d_{cen}(C_i, C_j)}\big)
$

- Dunn指数（Dunn Index，简称DI）

$
DI = \min\limits_{i\le i \le j} \big\{ \min\limits_{j\neq i} \big(\frac{d_{\min}(C_i,C_j)}{\max_{1 \le l \le k}diam(C_l)}\big)    \big\}
$

DBI的值越小越好，DI的值越大越好